In [1]:
import numpy as np 
import pandas as pd 


In [4]:
edges_df0 = pd.read_csv('projects/los_angeles/network_inputs/osm_edges_scag.csv')

day30 = pd.read_csv('projects/los_angeles/simulation_outputs/edges_df/edges_df_scenscag_5pct_bc30_r0_DYna_HR4_QT3.csv')

day90 = pd.read_csv('projects/los_angeles/simulation_outputs/edges_df/edges_df_scenscag_5pct_bc90_r0_DYna_HR4_QT3.csv')

day30.head()

,edge_id_igraph,type,tot_vol,true_vol,t_avg
0,0,motorway,9.0,0.0,38.37
1,1,motorway,9.0,0.0,258.21
2,6,motorway,2632.0,16.0,184.80
3,7,motorway,2615.0,16.0,47.42
4,8,motorway,2615.0,16.0,24.65


In [5]:
edges_df = pd.merge(edges_df0, day30[['edge_id_igraph', 'tot_vol']], how='left', on='edge_id_igraph')
edges_df = pd.merge(edges_df, day90[['edge_id_igraph', 'tot_vol']], how='left', on='edge_id_igraph', suffixes=['_day30', '_day90'])


In [6]:
edges_df = edges_df.fillna({'tot_vol_day30': 0, 'tot_vol_day90': 0})
edges_df['tot_vol_90-30'] = edges_df['tot_vol_day90'] - edges_df['tot_vol_day30']
print(np.max(edges_df['tot_vol_90-30']), np.min(edges_df['tot_vol_90-30']))

666.0 -340.0


In [7]:
edges_df.loc[edges_df['tot_vol_90-30']!=0, ['edge_id_igraph', 'tot_vol_day30', 'tot_vol_day90', 'tot_vol_90-30', 'geometry']].to_csv('projects/los_angeles/simulation_outputs/edge_vol_90-30.csv', index=False)

In [15]:
### process bridge closure
bc30 = pd.read_csv('projects/los_angeles/network_inputs/bridge_closure/bridge_closure_day30.csv')
bc30_osmid = bc30['OSMWayID1'].values.tolist() + bc30.dropna(subset=['OSMWayID2'])['OSMWayID2'].values.tolist()
bc30_df = edges_df.loc[edges_df['edge_osmid'].isin(bc30_osmid), ['edge_id_igraph', 'geometry']].to_csv('projects/los_angeles/simulation_outputs/30bc.csv', index=False)

bc90 = pd.read_csv('projects/los_angeles/network_inputs/bridge_closure/bridge_closure_day90.csv')
bc90_osmid = bc90['OSMWayID1'].values.tolist() + bc90.dropna(subset=['OSMWayID2'])['OSMWayID2'].values.tolist()

bc90_df = edges_df.loc[edges_df['edge_osmid'].isin(bc90_osmid), ['edge_id_igraph', 'geometry']].to_csv('projects/los_angeles/simulation_outputs/90bc.csv', index=False)